<a href="https://colab.research.google.com/github/guillerZeva/Bot-Agencia-de-Viajes-Telegram/blob/main/TravelBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalar librerias 

In [ ]:
!pip install python-telegram-bot
!pip install emoji
!apt install chromium-chromedriver
!pip install selenium

In [ ]:
import logging
from uuid import uuid4
from telegram import InlineQueryResultArticle, ParseMode, InputTextMessageContent, Update, ReplyKeyboardMarkup
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, ReplyKeyboardRemove,KeyboardButton
from telegram.ext import Updater, InlineQueryHandler, CommandHandler, CallbackContext, CallbackQueryHandler
from telegram.ext import ConversationHandler, MessageHandler,Filters
from telegram.utils.helpers import escape_markdown

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import  By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

from bs4 import BeautifulSoup as bs
import requests
import json
from emoji import emojize

from geopy.geocoders import Nominatim


options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

geolocalizador = Nominatim()
driver = webdriver.Chrome(chrome_options=options)

VALIDAR_NUMERO_ASIENTO, MOSTRAR_TICKET = range(2)
FECHA1,IDA_VUELO = range(2)
FECHA1IV,FECHA2,IDA_VUELO= range(3)

INPUT,OTRO_LUGAR,TICKET,COMPRAR = range(4)
ORIGEN,OTRO_LUGAR,DESTINO,IN_DESTINO,TICKET,COMPRAR= range(6)

vuelos_Dispo = []
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)



#------------------------------------------------------------------CLASES----------------------------------------------------------------------------------------------------

class Disponible:
  def __init__(self, lista_destino):
    self.lista_destino = lista_destino 

#VIAJES DISPONIBLES
class Viaje:
  pais_origen = ''
  pais_destino = ''
  origen = ''
  iataOrigen = ''
  destino = ''
  iataDestino = ''
  fechaInicial = ''
  fechaFinal = ''

  def __init__(self):
    pass


class Vuelo :
  fecha = ''
  pais_origen=''
  iata_origen = ''
  origen = ''
  aeropuerto_origen = ''
  pais_destion=''
  iata_destino = ''
  destino = ''
  aeropuerto_destino = ''
  aerolinea = ''
  hora_salida = ''
  etapas = ''
  hora_llegada = ''
  costo = ''
  def __init__(self):
    pass

class Vuelos_opciones:
  lista_vuelos = []
  def __init__(self):
    pass

class Ticket:
  vuelo = None
  cantidad_personas = 0  
  def __init__(self):
    pass

class User:
  tickets = []
  nombre = ''
  apellido = ''
  def __init__(self):
    pass

#-------------------------------------------------------------BUSCAR VIAJES----------------------------------------------------------------------------------------
def emoji_pais(pais):
  with open('paises.json', encoding='utf-8') as contenido:
    paises = json.load(contenido)
    valor ='' 
    for p in paises:
          if p['pais'] == pais:
            emoji = p['emoji']
            valor = emoji
  return valor



def Buscar_json_pais(iataCiudad):
  with open('IATA.json') as contenido:
    iatas = json.load(contenido)
    valor ='' 
    for pais in iatas:
      for i in iatas[pais]:

        if i['IATA'] == iataCiudad or iataCiudad in i['Ciudad'] or i['Ciudad'] in iataCiudad or iataCiudad == i['Aeropuerto']:
          valor = pais

  return valor


def lista_json(tipo):
  with open('IATA.json') as contenido:
    iatas = json.load(contenido)
    valor = [] 
    for pais in iatas:
      for i in iatas[pais]:
        if tipo == 'IATA':
          valor.append(i['IATA'])
        elif tipo == 'Ciudad':
          valor.append(i['Ciudad'])
        elif tipo == 'Aeropuerto':
          valor.append(i['Aeropuerto'])

  return valor

  
def Buscar_json(tipo,de,busca):

  retorno = lista_json(tipo)
  desde = lista_json(de)
  valor ='' 
  for b in zip (retorno,desde):
    if b[1] == busca:
      valor = b[0]

  return valor


def paises_viaje(viajes):
  paises = []
  for v in viajes:
    viaje = v.destino + " " + emojize(":"+emoji_pais(Buscar_json_pais(v.iataDestino))+":", use_aliases=True)
    paises.append(viaje)
    
  return paises


def buscar_vuelos_ida(origen, destino, fecha):

  url ='https://www.despegar.com.ec/shop/flights/results/oneway/'+origen+'/'+destino+'/'+fecha+'/1/0/0/NA/NA/NA/NA?from=SB&di=1-0'

  driver.get(url)
  time.sleep(1.5)
  c = 1
  lista = []
  while c < 5:
    texto = driver.find_elements_by_css_selector('#clusters > span:nth-child('+str(c)+') > span > cluster > div > div')
    if len(texto) ==0:
      break
    lista.append(texto)
    c += 1 

  list_vuelos = []
  for filto in lista:
    contenido = filto[0].text.replace(' \n', '').split('\n')
    c = 0
    viaje = []
    fil = ['meses sin intereses','¡Puedes sumar puntos' ,'Vuelo super flexible: podrás cambiar la fecha sin pagar multa.','Conocer más','IDA','Equipaje','¡Puedes sumar','Seleccionar','Ver opciones',
          'Llega más rápido, ¡Vuela directo!','Vuelo flexible: podrás cambiar la fecha sin pagar multa.','Ver bancos y tarjetas','Arma el vuelo a tu medida',
          'Ver bancos y tarjetas',  '  Únicamente en un pago', 'Precio final','Suma equipaje, selección de asiento y más', 'Clase']
    for i in contenido:
      if not i in fil:
        if not fil[0] in i:
          if not fil[1] in i:
            viaje.append(i)
      c += 1

      
    vuelo = Vuelo()
    vuelo.fecha = viaje[0]
    vuelo.iata_origen = viaje[1]
    vuelo.origen = viaje[2]
    vuelo.aeropuerto_origen = Buscar_json ('Aeropuerto','IATA',vuelo.iata_origen)
    vuelo.pais_origen = Buscar_json_pais(vuelo.iata_origen )
    vuelo.iata_destino = viaje[3]
    vuelo.destino = viaje[4]
    vuelo.aeropuerto_destino = Buscar_json('Aeropuerto','IATA',vuelo.iata_destino)
    vuelo.pais_destino = Buscar_json_pais(vuelo.iata_destino )
    vuelo.aerolinea = viaje[5]
    vuelo.hora_salida = viaje[6]
    vuelo.etapas = viaje[7]
    vuelo.hora_llegada = viaje[8]
    vuelo.costo = viaje[-1]

    list_vuelos.append(vuelo)

  return list_vuelos



def buscar_vuelos_ida_vuelta(origen, destino,fechaIni, fechaFinal):

  url ='https://www.despegar.com.ec/shop/flights/results/roundtrip/'+origen+'/'+destino+'/'+fechaIni+'/'+fechaFinal+'/1/0/0/NA/NA/NA/NA/NA?from=SB&di=1-0&reSearch=true'

  driver.get(url)
  time.sleep(1.5)
  c = 1
  lista = []
  while True:
    texto = driver.find_elements_by_css_selector('#clusters > span:nth-child('+str(c)+') > span > cluster > div > div')
    if len(texto) ==0:
      break
    lista.append(texto)
    c += 1 

  list_vuelos = []
  for filto in lista:
    contenido = filto[0].text.replace(' \n', '').split('\n')
    c = 0
    viaje = []
    fil = ['meses sin intereses','¡Puedes sumar puntos' ,'Vuelo super flexible: podrás cambiar la fecha sin pagar multa.','Conocer más','IDA','Equipaje','¡Puedes sumar','Seleccionar','Ver opciones',
          'Llega más rápido, ¡Vuela directo!','Vuelo flexible: podrás cambiar la fecha sin pagar multa.','Ver bancos y tarjetas','Arma el vuelo a tu medida',
          'Ver bancos y tarjetas',  '  Únicamente en un pago', 'Precio final','Suma equipaje, selección de asiento y más', 'Clase']
    for i in contenido:
      if not i in fil:
        if not fil[0] in i:
          if not fil[1] in i:
            viaje.append(i)
      c += 1

    vuelo = Vuelo()
    vuelo.fecha = viaje[0]
    vuelo.iata_origen = viaje[1]
    vuelo.origen = viaje[2]
    vuelo.aeropuerto_origen = Buscar_json ('Aeropuerto','IATA',vuelo.iata_origen)
    vuelo.pais_origen = Buscar_json_pais(vuelo.iata_origen )
    vuelo.iata_destino = viaje[3]
    vuelo.destino = viaje[4]
    vuelo.aeropuerto_destino = Buscar_json('Aeropuerto','IATA',vuelo.iata_destino)
    vuelo.pais_destino = Buscar_json_pais(vuelo.iata_destino )
    vuelo.aerolinea = viaje[5]
    vuelo.hora_salida = viaje[6]
    vuelo.etapas = viaje[7]
    vuelo.hora_llegada = viaje[8]
    vuelo.costo = viaje[-1]

    list_vuelos.append(vuelo)

  return list_vuelos


def contiene(vuelos,destino):
  no = True
  for i in vuelos:
    if i.destino == destino:
      no = False
      break
  return no

def buscar_viaje(ciudad):

  url ='https://www.latam.com/es-ec/vuelos-desde-'+ ciudad.lower()
  res = requests.get(url)
  soup = bs(res.content)
  lugares = soup.find_all('div', class_="card-body")
  vuelos = set ()

  for v in lugares:
    destino = v.find('span', class_='fare-atom-origin-destination-destination-city')
    fechas = v.find_all('p', class_='fare-atom-trip-leg-date')
    iataD = v.find('span', class_='fare-atom-origin-destination-destination-airport')

    if ( destino != None and contiene(vuelos,destino.get_text())):
      iata = iataD.get_text().replace(' ','').replace('(','').replace(')','')
      
      viaje = Viaje()
      
      viaje.pais_origen = Buscar_json_pais(ciudad) 
      viaje.pais_destino = Buscar_json_pais(destino.get_text()) 
      viaje.origen = ciudad
      viaje.iataOrigen = Buscar_json('IATA','Ciudad',ciudad)
      viaje.destino = destino.get_text()
      viaje.iataDestino = iata 
      viaje.fechaInicial = fechas[0].get_text()
      viaje.fechaFinal = fechas[1].get_text()

      vuelos.add(viaje)

  list_viajes = list(vuelos)
  return list_viajes

def enlistar_vuelos(vuelos):
  lista = []
  for v in vuelos:
    texto = v.iata_origen + ' -> ' + v.iata_destino +' a:'+v.aerolinea+' c:'+v.costo 
    lista.append(texto)

  return lista

lista = []
disponible = Disponible(lista)
viaje = Viaje()
vuelos_opciones = Vuelos_opciones()
vuelo = Vuelo()
usuario = User()
ticket = Ticket()
lista_inline_query_iata = lista_json('IATA')

lista_inline_query_ciudad = lista_json('Ciudad')

lista_inline_query_Aero = lista_json('Aeropuerto')

#-----------------------------------------------------------------------------------------------------------------------------------------------------
#POSICIONAR BOTONES EN COLUMNAS
def build_menu(buttons,n_cols,header_buttons=None,footer_buttons=None):
  menu = [buttons[i:i + n_cols] for i in range(0, len(buttons), n_cols)]
  if header_buttons:
    menu.insert(0, header_buttons)
  if footer_buttons:
    menu.append(footer_buttons)
  return menu

def mostrarBotones(lista,columns,callback):
  button_list = []
  for each in lista:

    button_list.append(InlineKeyboardButton(each, callback_data = each))
    

  reply_markup=InlineKeyboardMarkup(build_menu(button_list,n_cols=columns)) #n_cols = 1 is for single column and mutliple rows
  return reply_markup

#--------------------------------------------------------------------------------BOT---------------------------------------------------------------------
def start(update: Update, context: CallbackContext) -> None:
  print(update.message.chat)
  usuario.nombre = update.message.chat.first_name
  usuario.apellido = update.message.chat.last_name
  cnade = ''

  update.message.reply_text(f'¡Hola, {usuario.nombre} {emojize(":smile:", use_aliases=True)}!  \n'+
                              f'{emojize(":sparkles:", use_aliases=True)}¡Bienvenido a Travel Bot!{emojize(":sparkles:", use_aliases=True)}\n'+
                              '\nTravel bot te ayudará a buscar y reservar un vuelo'+ 
                              f' desde donde estés hacia cualquier parte del mundo {emojize(":airplane:", use_aliases=True)}{emojize(":earth_americas:", use_aliases=True)}.'+
                              f'\n usar /menu \n')
  

def menu(update: Update, context: CallbackContext) -> None:
  emoji_avion = emojize(":airplane:", use_aliases=True)
  emoji_buscar = emojize(":mag_right:", use_aliases=True)
  emoji_cod = emojize(":man_technologist:", use_aliases=True)

  vd = 'Vuelos Disponible'+' '+str(emoji_avion)
  b = 'Buscar'+' '+str(emoji_buscar)
  c = 'Código'+' '+str(emoji_cod)

  text_menu = f'{emojize(":sparkles:", use_aliases=True)} Menú de navegación {emojize(":sparkles:", use_aliases=True)}\n'
  
  botones_menu = [[InlineKeyboardButton(vd, callback_data = 'Vuelos_Disponible' )],
                  [InlineKeyboardButton(b, callback_data = 'buscar')],
                  [InlineKeyboardButton(c, url='https://github.com/guillerZeva/Bot-Agencia-de-Viajes-Telegram')]]

  reply_markup = InlineKeyboardMarkup(botones_menu)
  context.bot.send_message(chat_id=update.message.chat_id, text=text_menu ,reply_markup=reply_markup)


def botones_teclado(update, context):
  main_menu_keyboard = [[KeyboardButton(text= 'Menu',request_location = True)],
                          [KeyboardButton('Mi Agenda')]]
  reply_kb_markup = ReplyKeyboardMarkup(main_menu_keyboard,
                                                  resize_keyboard=True,
                                                  one_time_keyboard=True)

  

def escoger_origen(update,context,query):
  main_menu_keyboard = [[KeyboardButton(text= 'Ubicacion Actual',request_location = True)],
                          [KeyboardButton('Otro lugar')]]
  reply_kb_markup = ReplyKeyboardMarkup(main_menu_keyboard,
                                                   resize_keyboard=True,
                                                   one_time_keyboard=True)

    # Send the message with menu
  context.bot.send_message(chat_id=query.message.chat_id,
                     text='f',
                     reply_markup=reply_kb_markup) 
  

def lugar_Vuelos_Disponible(update,context):
  query = update.callback_query
  query.answer()

  escoger_origen(update,context,query)
  query.edit_message_text(text='ESCOGER lugar')

  return INPUT

def location(update,context):

  punto = str(update.message.location.latitude)+','+str(update.message.location.longitude)

  geolocalizador = Nominatim()
  direccion = geolocalizador.reverse(punto)
  aux = direccion.address.split(', ')
    
  return aux[3]

def viaje_disponible_location(update,context):
  loc = location(update,context)

  disponible.lista_destino = buscar_viaje(loc) 
  list_of_cities = paises_viaje(disponible.lista_destino)
  reply_markup = mostrarBotones(list_of_cities,1,'viajes')
  update.message.reply_text(text=f'Viajes Disponibles desde {loc}: ', reply_markup=reply_markup)
 
  return TICKET


#6° Pasaje 38A NO, Bastión Popular, Tarqui, Guayaquil, 090704, Ecuador

def viaje_disponible_otroLugar(update,context):
  update.message.reply_text('¡Hola,' )

  return OTRO_LUGAR

def viaje_disponible_input_lugar(update,context):
  
  disponible.lista_destino = buscar_viaje(update.message.text) 
  list_of_cities = paises_viaje(disponible.lista_destino)
  reply_markup = mostrarBotones(list_of_cities,1,'viajes')
  update.message.reply_text(text=f'Viajes Disponibles desde: {update.message.text}', reply_markup=reply_markup)
 
  return TICKET


def ticket(update,context):
  
  query = update.callback_query
  query.answer()

  list_of_cities = paises_viaje(disponible.lista_destino)

  for lugar in zip(list_of_cities, disponible.lista_destino):
    if update.callback_query.data == lugar[0]:
      
      viaje.origen = lugar[1].origen
      viaje.iataOrigen = lugar[1].iataOrigen
      viaje.destino = lugar[1].destino
      viaje.iataDestino  = lugar[1].iataDestino 


      emoji_ida = emojize(":ticket:", use_aliases=True)
      ti = emoji_ida + ' ' + 'Ticket ida'
      tiv = emoji_ida + ' ' + 'Ticket ida y vuelta'
      
      origen = emojize(":"+emoji_pais(lugar[1].pais_origen)+":", use_aliases=True) +' '+lugar[1].pais_origen
      destino = emojize(":"+emoji_pais(lugar[1].pais_destino)+":", use_aliases=True) +' '+lugar[1].pais_destino 

      text = origen +': '+ lugar[1].origen +' > > '+ destino +': ' +lugar[0]

      botones_menu = [[InlineKeyboardButton(ti, callback_data = 'ticket_ida' )],
                  [InlineKeyboardButton(tiv, callback_data = 'ticket_ida_vuelta')]]

      reply_markup = InlineKeyboardMarkup(botones_menu)
      query.edit_message_text(text=text , reply_markup=reply_markup)
  return COMPRAR

def ticket_ida(update,context):
  query = update.callback_query
  query.answer()
  query.edit_message_text(text='Escoger Fecha:' )
  context.bot.send_message(chat_id=update.callback_query.from_user.id,
                           text="Por favor escoger una fecha: ",
                          reply_markup=create_calendar())
  
  return FECHA1

def ticket_ida_vuelta(update,context):
  query = update.callback_query
  query.answer()
  query.edit_message_text(text='Escoger Fecha (Ida y vuelta):' )
  context.bot.send_message(chat_id=update.callback_query.from_user.id,
                           text="Por favor escoger una fecha: ",
                          reply_markup=create_calendar())
  
  return FECHA1IV


def press_button_callback(update, context): 
  query = update.callback_query
  query.answer()


  if update.callback_query.data == 'vuelos':
    print('0xcxzcAUIQ')





def inline_handler_ida(update, context):
    selected,date = process_calendar_selection(context.bot, update)
    if selected:

        reply_markup= InlineKeyboardMarkup([[InlineKeyboardButton(text="Back", callback_data="help_back")]])
        
        viaje.fechaInicial = date.strftime("%Y-%m-%d")

        print('viaje :z<<<<',viaje.origen,viaje.pais_destino , viaje.iataOrigen  , viaje.destino , viaje.iataDestino , viaje.fechaInicial )

        context.bot.send_message(chat_id=update.callback_query.from_user.id,
                        text="La fecha de partida es: %s" % (date.strftime("%d/%m/%Y")),
                        reply_markup=ReplyKeyboardRemove())
        
        selected,date = (False,None)
        co = 0
        vali = True
        while co < 3 and vali: 
          vuelos_opciones.lista_vuelos = buscar_vuelos_ida(viaje.iataOrigen,  viaje.iataDestino, viaje.fechaInicial)
          vali = False if len(vuelos_opciones.lista_vuelos) != 0 else True
          co += 1
        print(len(vuelos_opciones.lista_vuelos))
        reply_markup = mostrarBotones(enlistar_vuelos(vuelos_opciones.lista_vuelos),1,'vuelos')
        context.bot.send_message(chat_id=update.callback_query.from_user.id,
                             text=f'Vuelos de :',
                             reply_markup=reply_markup)  

    return IDA_VUELO  

def ida_vuelos (update, context):
    query = update.callback_query
    query.answer()
    print(update.callback_query.data)

    for vuelo in zip(enlistar_vuelos(vuelos_opciones.lista_vuelos), vuelos_opciones.lista_vuelos):
      if update.callback_query.data == vuelo[0]:
        ticket.vuelo = vuelo[1]
        query.edit_message_text('Numero de asientos: ')
    
    return VALIDAR_NUMERO_ASIENTO

def valida_numero_asiento (update, context):
  try:
    numero = int(update.message.text)
    update.message.reply_text('Bien')
    mostrar_ticket (update, context)
    return ConversationHandler.END
  except ValueError:
    update.message.reply_text('Mal')
    return VALIDAR_NUMERO_ASIENTO
    

def mostrar_ticket (update, context):
    print('algo ticket')
    update.message.reply_text(ticket.vuelo.aeropuerto_origen)

    return ConversationHandler.END



def inline_handler_idavuelta(update, context):
    selected,date = process_calendar_selection(context.bot, update)
    if selected:

        
        viaje.fechaInicial = date.strftime("%Y-%m-%d")

        context.bot.send_message(chat_id=update.callback_query.from_user.id,
                        text="La fecha de partida es: %s" % (date.strftime("%d/%m/%Y")))
        
        context.bot.send_message(chat_id=update.callback_query.from_user.id,
                        text="Regreso",
                        reply_markup = create_calendar())
      
       
    return FECHA2


def inline_handler2(update, context):
    selected,date = process_calendar_selection(context.bot, update)
    if selected:
        viaje.fechaFinal = date.strftime("%Y-%m-%d")

        context.bot.send_message(chat_id=update.callback_query.from_user.id,
                        text="La fecha de regreso es: %s" % (date.strftime("%d/%m/%Y")),
                        reply_markup=ReplyKeyboardRemove())
        
        print('viaje :-----------> ',viaje.origen, viaje.pais_destino, viaje.iataOrigen  , viaje.destino , viaje.iataDestino , viaje.fechaInicial , viaje.fechaFinal)
        selected,date = (False,None)
        co = 0
        vali = True
        while co < 3 and vali: 
          vuelos_opciones.lista_vuelos = buscar_vuelos_ida(viaje.iataOrigen,  viaje.iataDestino, viaje.fechaInicial)
          vali = False if len(vuelos_opciones.lista_vuelos) != 0 else True
          co += 1
        print(len(vuelos_opciones.lista_vuelos))
        reply_markup = mostrarBotones(enlistar_vuelos(vuelos_opciones.lista_vuelos),1,'vuelos')
        context.bot.send_message(chat_id=update.callback_query.from_user.id,
                             text=f'Vuelos de (ida vuelta):',
                             reply_markup=reply_markup)

    return IDA_VUELO


#--------------------------------------------------------------------------------------------------------------------------------------------------------
def buscar (update, context):
    query = update.callback_query
    query.answer()
    query.edit_message_text('Buscar:')
    escoger_origen(update,context,query)
    return ORIGEN

def buscar_location(update, context):
    loc = location(update,context)
    
    vuelo.pais_origen = Buscar_json_pais(loc)
    vuelo.origen = loc
    vuelo.iata_origen = Buscar_json('IATA','Ciudad',loc)
    vuelo.aeropuerto_origen = Buscar_json('Aeropuerto','IATA',vuelo.iata_origen)


    return DESTINO

def buscar_otroLugar(update, context): 

    update.message.reply_text('Ingrese por iata, aeropuerto, ciudad:\n @TravelBot iata \n @TravelBot ciudad\n @TravelBot airport' )

    return OTRO_LUGAR

def tran_iata(text):
    iata = ''
    c = text

    if len(c) > 15:
      iata = Buscar_json('IATA','Aeropuerto',c)
    elif len(c) != 3:
      iata = Buscar_json('IATA','Ciudad',c)
    else:
      iata =  c
    return iata 

def buscar_input_lugar(update, context): 

    iata = tran_iata(update.message.text)
    
    vuelo.pais_origen = Buscar_json_pais(iata)
    vuelo.origen = Buscar_json('Ciudad','IATA',iata)
    vuelo.iata_origen = iata
    vuelo.aeropuerto_origen = Buscar_json('Aeropuerto','IATA',iata)

    return DESTINO
    
def buscar_destino(update, context): 
    update.message.reply_text('Ingrese Destino por iata, aeropuerto, ciudad:\n @TravelBot iata \n @TravelBot ciudad\n @TravelBot airport')

    return IN_DESTINO

def buscar_input_destion(update, context): 

    iata = tran_iata(update.message.text)
    vuelo.pais_destino = Buscar_json_pais(iata)
    vuelo.destino = Buscar_json('Ciudad','IATA',iata)
    vuelo.iata_destino = iata
    vuelo.aeropuerto_destino = Buscar_json('Aeropuerto','IATA',iata)
    return TICKET

def buscar_ticket(update, context): 

    emoji_ida = emojize(":ticket:", use_aliases=True)
    ti = emoji_ida + ' ' + 'Ticket ida'
    tiv = emoji_ida + ' ' + 'Ticket ida y vuelta'
      
    origen = emojize(":"+emoji_pais(vuelo.pais_origen)+":", use_aliases=True) +' '+vuelo.pais_origen
    destino = emojize(":"+emoji_pais(vuelo.pais_destino)+":", use_aliases=True) +' '+vuelo.pais_destino 

    text = origen +': '+ vuelo.origen +' > > '+ destino +': ' +vuelo.destino

    botones_menu = [[InlineKeyboardButton(ti, callback_data = 'ticket_ida' )],
                  [InlineKeyboardButton(tiv, callback_data = 'ticket_ida_vuelta')]]

    reply_markup = InlineKeyboardMarkup(botones_menu)
    update.message.reply_text(text=text , reply_markup=reply_markup)

    return COMPRAR

def buscar_porIATA(update, context): 
    query = update.inline_query.query
    
    results = []
    for m in lista_inline_query_iata:
      results.append(InlineQueryResultArticle(
                    id=uuid4(),
                    title=m,
                    input_message_content=InputTextMessageContent(m)))
    
    update.inline_query.answer(results) 

def buscar_porCiudad(update, context): 
    query = update.inline_query.query

    results = []
    for m in lista_inline_query_ciudad:
      results.append(InlineQueryResultArticle(
                    id=uuid4(),
                    title=m,
                    input_message_content=InputTextMessageContent(m)))
    
    update.inline_query.answer(results) 

def buscar_porAero(update, context): 
    query = update.inline_query.query

    results = []
    for m in lista_inline_query_Aero:
      results.append(InlineQueryResultArticle(
                    id=uuid4(),
                    title=m,
                    input_message_content=InputTextMessageContent(m)))
    
    update.inline_query.answer(results) 


def cancel(update, context):

  print(update.message.text)
  lista = []
  disponible = Disponible(lista)
  viaje = Viaje()
  vuelos_opciones = Vuelos_opciones()
  ticket = Ticket()


  main_menu_keyboard = [[KeyboardButton(text= '/start')],
                          [KeyboardButton('Mi Agenda')]]
  reply_kb_markup = ReplyKeyboardMarkup(main_menu_keyboard,
                                                  resize_keyboard=True,
                                                  one_time_keyboard=True)

  update.message.reply_text('Accion cancelada',reply_markup=reply_kb_markup)

  return ConversationHandler.END

def main() -> None:

  updater = Updater("1605195949:AAGt-qZLD7xEclG5MHrAymIui85IMq9LZic")

  dispatcher = updater.dispatcher

  dispatcher.add_handler(CommandHandler("start", start))

  dispatcher.add_handler(CommandHandler("menu", menu))

  dispatcher.add_handler(CommandHandler("cancel", cancel))

  compra_ticket_handler = ConversationHandler(
  entry_points=[CallbackQueryHandler(callback= ida_vuelos)],
  states={
      VALIDAR_NUMERO_ASIENTO: [MessageHandler(Filters.text, valida_numero_asiento)],
      MOSTRAR_TICKET : [MessageHandler(Filters.text, mostrar_ticket)]
  },
  fallbacks=[CommandHandler("cancel", cancel),
             CommandHandler("menu", menu)]
  )


  
  ticket_ida_handler = ConversationHandler(
  entry_points=[CallbackQueryHandler(callback= ticket_ida, pattern='ticket_ida')],
  states={
      FECHA1: [CallbackQueryHandler(inline_handler_ida)],
      IDA_VUELO: [compra_ticket_handler]
  },
  fallbacks=[CommandHandler("cancel", cancel),
             CommandHandler("menu", menu)]
  )


  ticket_ida_vuelta_handler = ConversationHandler(
  entry_points=[CallbackQueryHandler(callback= ticket_ida_vuelta, pattern='ticket_ida_vuelta')],
  states={
      FECHA1IV:  [CallbackQueryHandler(inline_handler_idavuelta)],
      FECHA2:   [CallbackQueryHandler(inline_handler2)],
      IDA_VUELO: [compra_ticket_handler]

  },
  fallbacks=[CommandHandler("cancel", cancel),
             CommandHandler("menu", menu)]
  )
  

  vuelos_handler = ConversationHandler(
  entry_points=[CallbackQueryHandler(callback= lugar_Vuelos_Disponible, pattern='Vuelos_Disponible')],
  states={
      INPUT: [MessageHandler(Filters.text, viaje_disponible_otroLugar),MessageHandler(Filters.location, viaje_disponible_location)],
      OTRO_LUGAR:  [MessageHandler(Filters.text, viaje_disponible_input_lugar)],
      TICKET: [CallbackQueryHandler(callback= ticket)],
      COMPRAR:[ticket_ida_vuelta_handler,
               ticket_ida_handler],

  },
  fallbacks=[CommandHandler("cancel", cancel),
             CommandHandler("menu", menu)]
  )

  buscar_handler = ConversationHandler(
  entry_points=[CallbackQueryHandler(callback= buscar, pattern='buscar')],
  states={
      ORIGEN: [MessageHandler(Filters.text, buscar_otroLugar),MessageHandler(Filters.location, buscar_location)],
      OTRO_LUGAR: [MessageHandler(Filters.text, buscar_input_lugar)],
      DESTINO: [MessageHandler(Filters.text,  buscar_destino)],
      IN_DESTINO: [MessageHandler(Filters.text, buscar_input_destion)],
      TICKET: [MessageHandler(Filters.text, buscar_ticket)],
      COMPRAR:[ticket_ida_vuelta_handler,
            ticket_ida_handler],

  },
  fallbacks=[CommandHandler("cancel", cancel),
             CommandHandler("menu", menu)]
  )


  dispatcher.add_handler(vuelos_handler,1)

  dispatcher.add_handler(buscar_handler,2)

  dispatcher.add_handler(CallbackQueryHandler(callback = press_button_callback))

  dispatcher.add_handler(InlineQueryHandler(buscar_porIATA, pattern='iata'))
  
  dispatcher.add_handler(InlineQueryHandler(buscar_porCiudad, pattern='ciudad'))
  
  dispatcher.add_handler(InlineQueryHandler(buscar_porAero, pattern='airport'))


  updater.start_polling()

  updater.idle()

if __name__ == '__main__':
  
  main()




/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.7/dist-packages/telegram/ext/conversationhandler.py:280: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  "If 'per_message=False', 'CallbackQueryHandler' will not be "
2021-03-04 05:58:49,562 - apscheduler.scheduler - INFO - Scheduler started
/usr/loc

viaje :----------->  Guayaquil  GYE Buenos Aires BUE 2021-03-19 2021-03-21
4
GYE -> EZE a:LATAM c:USD378
algo ticket


2021-03-04 06:01:14,993 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-03-04 06:01:14,995 - apscheduler.scheduler - INFO - Scheduler has been shut down
2021-03-04 06:01:15,161 - telegram.ext.updater - WARNING - Exiting immediately!


#CALENDARIO 

In [1]:
from telegram import InlineKeyboardButton, InlineKeyboardMarkup,ReplyKeyboardRemove
import datetime
import calendar

def create_callback_data(action,year=0,month=0,day=0):
    return ";".join([action,str(year),str(month),str(day)])

def separate_callback_data(data):
    lis = data.split(";")
    if lis[0] != 'DAY' and lis[0] != 'IGNORE' and lis[0] != 'PREV-MONTH' and lis[0] != 'NEXT-MONTH':
      lis = ['IGNORE',2020,1,0]

    return lis


def create_calendar(year=None,month=None):

    now = datetime.datetime.now()
    if year == None: year = now.year
    if month == None: month = now.month
    data_ignore = create_callback_data("IGNORE", year, month, 0)
    keyboard = []
    #First row - Month and Year
    row=[]
    row.append(InlineKeyboardButton(calendar.month_name[month]+" "+str(year),callback_data=data_ignore))
    keyboard.append(row)
    #Second row - Week Days
    row=[]
    for day in ["Mo","Tu","We","Th","Fr","Sa","Su"]:
        row.append(InlineKeyboardButton(day,callback_data=data_ignore))
    keyboard.append(row)

    my_calendar = calendar.monthcalendar(year, month)
    for week in my_calendar:
        row=[]
        for day in week:
            if(day==0):
                row.append(InlineKeyboardButton(" ",callback_data=data_ignore))
            else:
                row.append(InlineKeyboardButton(str(day),callback_data=create_callback_data("DAY",year,month,day)))
        keyboard.append(row)
    #Last row - Buttons
    row=[]
    row.append(InlineKeyboardButton("<",callback_data=create_callback_data("PREV-MONTH",year,month,day)))
    row.append(InlineKeyboardButton(" ",callback_data=data_ignore))
    row.append(InlineKeyboardButton(">",callback_data=create_callback_data("NEXT-MONTH",year,month,day)))
    keyboard.append(row)

    return InlineKeyboardMarkup(keyboard)


def process_calendar_selection(bot,update):

    ret_data = (False,None)
    query = update.callback_query

    (action,year,month,day) = separate_callback_data(query.data)
    curr = datetime.datetime(int(year), int(month), 1)
    if action == "IGNORE":
        bot.answer_callback_query(callback_query_id= query.id)
    elif action == "DAY":
        bot.edit_message_text(text=query.message.text,
            chat_id=query.message.chat_id,
            message_id=query.message.message_id
            )
        ret_data = True,datetime.datetime(int(year),int(month),int(day))
    elif action == "PREV-MONTH":
        pre = curr - datetime.timedelta(days=1)
        bot.edit_message_text(text=query.message.text,
            chat_id=query.message.chat_id,
            message_id=query.message.message_id,
            reply_markup=create_calendar(int(pre.year),int(pre.month)))
    elif action == "NEXT-MONTH":
        ne = curr + datetime.timedelta(days=31)
        bot.edit_message_text(text=query.message.text,
            chat_id=query.message.chat_id,
            message_id=query.message.message_id,
            reply_markup=create_calendar(int(ne.year),int(ne.month)))
    else:
        bot.answer_callback_query(callback_query_id= query.id,text="Algo salió mal!")
        # UNKNOWN
    return ret_data